In [169]:
from datetime import datetime
import pandas as pd
import pandas_datareader.data as web

def getStock(Stock):
    stock_df = web.DataReader(Stock, 'yahoo', start_time, now_time)
    return stock_df
if __name__ == '__main__':
    now_time = datetime.now()
    start_time = datetime(now_time.year - 8, now_time.month, now_time.day)
    stock_df = getStock('TSLA')


In [170]:
stock_df.dropna()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-05-27,42.773998,41.540001,41.703999,42.312000,26705500.0,42.312000
2014-05-28,42.554001,41.051998,42.004002,42.048000,27475500.0,42.048000
2014-05-29,42.498001,41.543999,42.113998,42.048000,18462500.0,42.048000
2014-05-30,42.959999,41.403999,42.060001,41.554001,27905500.0,41.554001
2014-06-02,41.869999,40.334000,41.466000,40.939999,23340500.0,40.939999
...,...,...,...,...,...,...
2022-05-18,760.500000,700.809998,744.520020,709.809998,29270600.0,709.809998
2022-05-19,734.000000,694.109985,707.000000,709.419983,30098900.0,709.419983
2022-05-20,721.580017,633.000000,713.989990,663.900024,48190600.0,663.900024


In [171]:
stock_df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-05-27,42.773998,41.540001,41.703999,42.312000,26705500.0,42.312000
2014-05-28,42.554001,41.051998,42.004002,42.048000,27475500.0,42.048000
2014-05-29,42.498001,41.543999,42.113998,42.048000,18462500.0,42.048000
2014-05-30,42.959999,41.403999,42.060001,41.554001,27905500.0,41.554001
2014-06-02,41.869999,40.334000,41.466000,40.939999,23340500.0,40.939999


In [172]:
import numpy as np
from scipy.stats import norm
import scipy.optimize as opt
import pandas as pd
import datetime
import time
from arch import arch_model
import matplotlib.pyplot as plt
from numba import jit
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
import warnings
import plotly.express as px
import plotly.graph_objects as go
import math
from datetime import timedelta
warnings.filterwarnings('ignore')

In [175]:
# stock_df.reset_index(inplace=True)
stock_df.sort_values(by=['Date'],ascending=False)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-05-24,653.750000,620.570129,653.530029,628.159973,29229670.0,628.159973
2022-05-23,679.960022,638.059998,655.020020,674.900024,29634500.0,674.900024
2022-05-20,721.580017,633.000000,713.989990,663.900024,48190600.0,663.900024
2022-05-19,734.000000,694.109985,707.000000,709.419983,30098900.0,709.419983
2022-05-18,760.500000,700.809998,744.520020,709.809998,29270600.0,709.809998
...,...,...,...,...,...,...
2014-06-02,41.869999,40.334000,41.466000,40.939999,23340500.0,40.939999
2014-05-30,42.959999,41.403999,42.060001,41.554001,27905500.0,41.554001
2014-05-29,42.498001,41.543999,42.113998,42.048000,18462500.0,42.048000


In [176]:
ret = 100 * (stock_df.pct_change()[1:]['Adj Close'])
realized_vol = ret.rolling(5).std()

In [119]:
ret

Date
2014-05-28   -0.623936
2014-05-29    0.000000
2014-05-30   -1.174847
2014-06-02   -1.477601
2014-06-03    0.117246
                ...   
2022-05-18   -6.801380
2022-05-19   -0.054946
2022-05-20   -6.416504
2022-05-23    1.656876
2022-05-24   -5.514823
Name: Adj Close, Length: 2014, dtype: float64

In [177]:
newnames = {'Adj Close':'Realized Volatility'} # For Visualizations
fig = px.line(realized_vol, template='plotly_white', labels={'value':'Volatility'}, title='Realized Volatility')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.update_layout(
    title_x=0.5,
    legend_title="Legend"
)
fig.show()

In [178]:
period_volatility = {}
daily_volatility = ret.std()
print('Daily volatility: ', '{:.2f}%'.format(daily_volatility))
period_volatility['Daily Volatility'] = round(daily_volatility, 2)

monthly_volatility = math.sqrt(21) * daily_volatility
print ('Monthly volatility: ', '{:.2f}%'.format(monthly_volatility))
period_volatility['Monthly Volatility'] = round(monthly_volatility, 2)

annual_volatility = math.sqrt(252) * daily_volatility
print ('Annual volatility: ', '{:.2f}%'.format(annual_volatility ))
period_volatility['Annual Volatility'] = round(annual_volatility, 2)

Daily volatility:  3.47%
Monthly volatility:  15.89%
Annual volatility:  55.05%


In [179]:
n = 300
split_date = ret.iloc[-n:].index

In [180]:
retv = ret.values

fig = px.line(x=stock_df.index[1:], y=ret, template='plotly_white', labels={'x':'Date', 'y':'Daily Returns'}, 
              title='Volatility clustering')
fig.update_layout(
    title_x=0.5
)
fig.show()

# SVR-GARCH

In [181]:
from sklearn.svm import SVR
from scipy.stats import uniform as sp_rand
from sklearn.model_selection import RandomizedSearchCV

realized_vol = ret.rolling(5).std() 
realized_vol = pd.DataFrame(realized_vol)
realized_vol.reset_index(drop=True, inplace=True)

In [182]:
returns_svm = ret ** 2
returns_svm = returns_svm.reset_index()
del returns_svm['Date']

In [183]:
X = pd.concat([realized_vol, returns_svm], axis=1, ignore_index=True)
X = X[4:].copy()
X = X.reset_index()
X.drop('index', axis=1, inplace=True)

In [184]:
X.iloc[:-n].values

array([[7.01886886e-01, 1.37465242e-02],
       [7.05990786e-01, 2.14875792e-01],
       [1.15464790e+00, 2.03502293e+00],
       ...,
       [8.53088379e+00, 4.19574004e+00],
       [3.42800127e+00, 1.92490778e+01],
       [3.69257506e+00, 1.35650354e+01]])

In [185]:
realized_vol = realized_vol.dropna().reset_index()
realized_vol.drop('index', axis=1, inplace=True)

In [187]:
svr_lin = SVR(kernel='linear') 

In [188]:
para_grid = {'gamma': sp_rand(), 'C': sp_rand(), 'epsilon': sp_rand()}
clf = RandomizedSearchCV(svr_lin, para_grid)
clf.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,))



RandomizedSearchCV(estimator=SVR(kernel='linear'),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E58A2420D0>,
                                        'epsilon': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E5879C47C0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E58A23A4C0>})

In [189]:
predict_svr_lin = clf.predict(X.iloc[-n:])

predict_svr_lin = pd.DataFrame(predict_svr_lin)
predict_svr_lin.index = ret.iloc[-n:].index

rmse_svr = np.sqrt(mse(realized_vol.iloc[-n:] / 100, predict_svr_lin / 100))
print('The RMSE value of SVR with Linear Kernel is {:.6f}'.format(rmse_svr))
mse_svr = mse(realized_vol.iloc[-n:] / 100, predict_svr_lin / 100)
print('The MSE value of SVR with Linear Kernel is {:.7f}'.format(mse_svr))

mae_svr = mae(realized_vol.iloc[-n:] / 100, predict_svr_lin / 100)
print('The MAE value of SVR with Linear Kernel is {:.6f}'.format(mae_svr))

The RMSE value of SVR with Linear Kernel is 0.001970
The MSE value of SVR with Linear Kernel is 0.0000039
The MAE value of SVR with Linear Kernel is 0.001481


In [190]:
predict_svr_lin

,0
Date,
2021-03-18,4.314228
2021-03-19,4.174458
2021-03-22,4.235923
2021-03-23,3.882032
2021-03-24,3.661508
...,...
2022-05-18,5.606060
2022-05-19,5.425649
2022-05-20,5.000941


In [191]:
realized_vol.index = ret.iloc[4:].index

svr_lin_forecast_df = predict_svr_lin / 100
svr_lin_forecast_df.columns = ['Preds']
fig = px.line(realized_vol / 100, template='plotly_white', labels={'value':'Volatility'}, title='Volatility Prediction with SVR-GARCH (Linear)')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.add_trace(go.Scatter(x=svr_lin_forecast_df.index, y=svr_lin_forecast_df['Preds'], mode='lines', name='SVR-GARCH Predictions'))
fig.update_layout(
    title_x=0.5,
    legend_title="Legend"
)
fig.show()

In [192]:
from datetime import date
dt = date.today()
svr_lin_forecast_df.loc[[dt]].values[0].item()

0.04108036386049543

# Neural Networks

In [193]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [194]:
model = keras.Sequential(
    [layers.Dense(256, activation="relu"),
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="linear"),])

In [195]:
model.compile(loss='mse', optimizer='rmsprop')

In [196]:
epochs_trial = np.arange(100, 400, 4)
batch_trial = np.arange(100, 400, 4)

In [197]:

DL_pred = []
DL_RMSE = []
DL_MSE = []
DL_MAE = []
for i, j, k in zip(range(4), epochs_trial, batch_trial):
    model.fit(X.iloc[:-n].values, realized_vol.iloc[1:-(n-1)].values.reshape(-1,),
              batch_size=k, epochs=j, verbose=False)
    DL_predict = model.predict(np.asarray(X.iloc[-n:]))
    DL_RMSE.append(np.sqrt(mse(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100)))
    DL_MSE.append(mse(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100))
    DL_MAE.append(mae(realized_vol.iloc[-n:] / 100, DL_predict.flatten() / 100))
    DL_pred.append(DL_predict)
    print('DL_RMSE_{}:{:.6f}'.format(i+1, DL_RMSE[i]))     
    print('DL_MSE_{}:{:.6f}'.format(i+1, DL_MSE[i]))    
    print('DL_MAE_{}:{:.6f}'.format(i+1, DL_MAE[i]))    


10/10 [==============================] - 0s 969us/step
DL_RMSE_1:0.003092
DL_MSE_1:0.000010
DL_MAE_1:0.002468
10/10 [==============================] - 0s 442us/step
DL_RMSE_2:0.003788
DL_MSE_2:0.000014
DL_MAE_2:0.002404
10/10 [==============================] - 0s 2ms/step
DL_RMSE_3:0.003964
DL_MSE_3:0.000016
DL_MAE_3:0.003039
10/10 [==============================] - 0s 560us/step
DL_RMSE_4:0.005074
DL_MSE_4:0.000026
DL_MAE_4:0.003847


In [198]:
DL_predict = pd.DataFrame(DL_pred[DL_RMSE.index(min(DL_RMSE))])
DL_predict.index = ret.iloc[-n:].index

In [199]:
nn_forecast_df = DL_predict / 100
nn_forecast_df.columns = ['Preds']
fig = px.line(realized_vol / 100, template='plotly_white', labels={'value':'Volatility'}, title='Volatility Prediction with Neural Networks')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.add_trace(go.Scatter(x=nn_forecast_df.index, y=nn_forecast_df['Preds'], mode='lines', name='Neural Network Predictions'))
fig.update_layout(
    title_x=0.5,
    legend_title="Legend"
)
fig.show()

In [200]:
from datetime import date
dt = date.today()
nn_forecast_df.loc[[dt]].values[0].item()

0.043280307203531265

# HAR-Realized Volatility

In [201]:
rv_daily = ret.rolling(2).std()
rv_weekly = ret.rolling(5).std()
rv_monthly = ret.rolling(22).std()

In [216]:
rv_df = pd.DataFrame()
rv_df['Daily'] = rv_daily
rv_df['Weekly'] = rv_weekly
rv_df['Month'] = rv_monthly
rv_df = rv_df.dropna()
rv_df

,Daily,Weekly,Month
Date,,,
2014-06-26,1.720196,2.066697,2.381716
2014-06-27,1.423511,2.096685,2.375776
2014-06-30,0.742665,1.568799,2.372319
2014-07-01,0.395937,1.037460,2.344942
2014-07-02,2.935110,2.188727,2.537069
...,...,...,...
2022-05-18,8.444550,5.897573,5.099727
2022-05-19,4.770449,5.898081,5.039166
2022-05-20,4.498300,5.223651,5.095533


In [218]:
import statsmodels.api as sm

rv_df["Target"] = rv_df["Daily"]
rv_df.dropna(inplace = True)

# #Scale the data
rv_scaled = (rv_df-rv_df.min())/(rv_df.max()-rv_df.min())

# #Add constant c
rv_scaled = sm.add_constant(rv_scaled)

#Split train and test sets and fit to the model
X = rv_scaled.drop("Target", axis = 1)
y = rv_scaled["Target"]
split = len(X) - n
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [219]:
rv_df

,Daily,Weekly,Month,Target
Date,,,,
2014-06-26,1.720196,2.066697,2.381716,1.720196
2014-06-27,1.423511,2.096685,2.375776,1.423511
2014-06-30,0.742665,1.568799,2.372319,0.742665
2014-07-01,0.395937,1.037460,2.344942,0.395937
2014-07-02,2.935110,2.188727,2.537069,2.935110
...,...,...,...,...
2022-05-18,8.444550,5.897573,5.099727,8.444550
2022-05-19,4.770449,5.898081,5.039166,4.770449
2022-05-20,4.498300,5.223651,5.095533,4.498300


In [220]:
results = sm.OLS(y_train, X_train).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Target   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.744e+33
Date:                Tue, 24 May 2022   Prob (F-statistic):               0.00
Time:                        17:53:23   Log-Likelihood:                 61278.
No. Observations:                1693   AIC:                        -1.225e+05
Df Residuals:                    1689   BIC:                        -1.225e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.036e-17   2.03e-18     14.944      0.000    2.64e-17    3.43e-17
Daily          1.0000   1.43e-17   7.01e+16      0.000       1.000       1.000
Weekly       3.99e-17   1.35e-17      2.947      0.003    1.33e-17    6.65e-17
Month       1.041e-17    8.9e-18      1.170      0.242   -7.04e-18    2.79e-17
==============================================================================
Omnibus:                     1385.830   Durbin-Watson:                   1.310
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            46148.560
Skew:                           3.621   Prob(JB):                         0.00
Kurtosis:                      27.531   Cond. No.                         16.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [221]:
y_hat = results.predict(X_test)

denormalized_y = y_hat * (max(y_test) - min(y_test)) + min(y_test)

rmse_har_rv = np.sqrt(mse(realized_vol.iloc[-n:] / 100, denormalized_y))
print('The RMSE value of HAR-RV Model is  {:.6f}'.format(rmse_har_rv))

mse_har_rv = mse(realized_vol.iloc[-n:] / 100, denormalized_y)
print('The MSE value of HAR-RV Model is  {:.6f}'.format(mse_har_rv))

mae_har_rv = mae(realized_vol.iloc[-n:] / 100, denormalized_y)
print('The MAE value of HAR-RV Model is  {:.6f}'.format(mae_har_rv))

fig = px.line(realized_vol / 100, template='plotly_white', labels={'value':'Volatility'}, title='Volatility Prediction with HAR-RV Model')
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.add_trace(go.Scatter(x=y_test.index, y=denormalized_y, mode='lines', name='HAR-RV Predictions'))
fig.update_layout(
    title_x=0.5,
    legend_title="Legend"
)
fig.show()

The RMSE value of HAR-RV Model is  0.050666
The MSE value of HAR-RV Model is  0.002567
The MAE value of HAR-RV Model is  0.036841


In [232]:
str_dt = dt.strftime("%Y-%m-%d")
print(y_hat.get(key=str_dt))

0.24932618344338217
